#### The Objective

The ultimate aim of of the challenge is to **predict the area of wildfires in 7 regions in Australia for February 2021** with historical data, before they have happened! 

There are three submissions:
- 1) Predict wildfires in February 2020.
- 2) Predict wildifres in 3rd and 4th week of January 2021.
- 3) Predict wildfires in February 20201.

#### 1.2 Historical Weather

This dataset contains daily aggregates computed from the hourly ERA5 climate reanalysis. Find more information about this data [here](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview) and [here](https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5)

#### Variables

* All variables are aggregated to daily values from `YYYY-mm-ddT01:00:00Z` to `YYYY-mm-(dd+1)T00:00:00Z`
* `Precipitation` is derived from total precipitation. Hourly raw data is converted from m/hour to mm/hour 
* [`Relative humidity`](https://en.wikipedia.org/wiki/Relative_humidity) is derived from the temperature and dewpoint
* `Soil water content` is given for 0 - 7 cm below the surface
* `Solar radiation`or Surface Solar Radiation Downwards. Units are converted from J/h to MJ/h
* `Temperature`
* `Wind speed` is calculated for every hour from the Easterly and Northerly 10 meter wind components

#### Steps:
[1. Load Packages](#LoadPackages) 

[2. Descriptive Stats](#DescriptiveStats) 

[3. Evaluating for Missing Values(no missing values)](#MissingValues) 

[4. Checking for Duplicates (no duplicates)](#Duplicates) 

[5. Rearranging Table via Pivot](#PivotTable) 

[6. Evaluate Re-Arranced Parameter Columns for Missing and Duplicates](#RearrangedTable) 

[7. Weather Data Review](#DataReview) 

[8. Save out Pre-Processed "C&P_Weather" CSV File](#PreprocessedWeather) 

#### Load packages <a class="anchor" id="LoadPackages"></a>

In [ ]:
# Import the necessary packages for analysis and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt, mpld3
%matplotlib inline
import json
import datetime

from shapely.geometry import Polygon, mapping
import geopandas as gpd
import folium
from folium.plugins import TimeSliderChoropleth
import seaborn as sns
import plotly.express as px

sns.set_style("whitegrid")

import warnings
warnings.filterwarnings("ignore")

#### Notes:

* Data type has been changed to match across all other datasets.
* Renaming columns to make more sense.
* No null values.
* No duplicates or drops.

In [ ]:
# Load the dataset
weather = "H_Weather.csv"
print("Reading file: '{}'".format(weather))
weather_df = pd.read_csv(weather, parse_dates=[1])
print("Loaded...")

# Columns and their datatypes
print(weather_df.dtypes)

weather_df.head()

#### Notes:
For every region {object}:

    1 - Date : here is an object and will need to be defined as (format YYYY-MM-DD) {datetime64[ns]}
    2 - Parameter includes: {object}

            Precipiation (mmd/day)
            Relative Humidity (%)
            Soil water content (m3 m3)
            Solar Radiation (MJ/day)
            Temperature (C)
            Wind speed (m/s)

    3 - Count - (km2) {float64}

In [ ]:
#changing date type for consistency across all datasets
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df.head()

In [ ]:
# rename columns
weather_cols = ['Date', 'Region', 'Parameter', 'area', 'min', 'max', 'mean', '2nd_moment']
weather_df.columns= weather_cols

In [ ]:
weather_df.columns.tolist()

#### Descriptive Stats <a class="anchor" id="DescriptiveStats"></a>

In [ ]:
weather_df.dtypes

In [ ]:
weather_df.shape

In [ ]:
weather_df.info()

#### Evaluating for Missing Values <a class="anchor" id="MissingValues"></a>

In [ ]:
# check for missing values
weather_df.isna().sum()

#### Checking for Duplicates <a class="anchor" id="Duplicates"></a>

In [ ]:
# find duplicates
weather_df.duplicated().sum()

In [ ]:
weather_df.describe().transpose()

In [ ]:
print("Rows    : ", weather_df.shape[0])
print("Columns : ", weather_df.shape[1])
print("\nFeatures : ", weather_df.columns.tolist())
print("\nMissing Values : \n",weather_df.isnull().any())
print("\nUnique Values : \n",weather_df.nunique())
print("Number of records: {}".format(len(weather_df)))
print("Number of regions: {}\n".format(len(weather_df['Region'].unique())))
print(weather_df['Region'].unique())
print(weather_df['Parameter'].unique())

#### Re-arranging Table via Pivot Function <a class="anchor" id="PivotTable"></a>

In [ ]:
# Rename columns names
weather_df.columns = ['Date', 'Region', 'Parameter', 'area', 'min', 'max', 'mean', '2nd_moment']
weather_df.head()

In [ ]:
#rearranging Paramater values in the weather data
df_pivot = weather_df.pivot_table(values=['min','max','mean','2nd_moment'], index=['Date','Region', 'area'], columns=['Parameter'])
df_pivot

In [ ]:
#resetting the index on the new table formed
df_pivot.reset_index(inplace=True)
df_pivot.head()

In [ ]:
# Renaming Column names
df_pivot.columns = [col[0] if not(col[1]) else '{1}_{0}'.format(*col) for col in df_pivot.columns.values]
df_pivot.head()

In [ ]:
# Rearranging Data and column
params = df_pivot.columns.tolist()[3:]
params.sort()
weather_data = df_pivot[df_pivot.columns.tolist()[:3] + params].copy()
weather_data.head()

In [ ]:
num_rows, num_cols = weather_data.shape
print("There are total {} records in the following {} columns:\n".format(num_rows, num_cols))
print("\n".join(list(weather_data.columns)))

#### Evaluate Re-Arranged Paramater Columns for Missing and Duplicates <a class="anchor" id="RearrangedTable"></a>

Note: Check for null values in the weather data paramater columns now.

In [ ]:
weather_data.isna().sum()

Checking NULL values for - PRECIPITATION

In [ ]:
#cross checking null values in the new arranged data for the Precipitation column
weather_data.loc[weather_data['Precipitation_mean'].isna(), :]

In [ ]:
#verifying the original data, that indeed there was no rain on 06-22-2019
weather_df.loc[weather_df['Date'] == "2019-06-22", :]

Checking NULL values for - TEMPERATURE

In [ ]:
weather_data.loc[weather_data['Temperature_mean'].isna(), :]

Checking min value in the original data, as it appears to be null for two dates 02/08/2009 and 06/19/2018

In [ ]:
weather_df.loc[weather_df['Date'] == "2009-02-08", :]

In [ ]:
weather_df.loc[weather_df['Date'] == "2018-06-19", :]

This confirms that Temprature is null for two dates 2009-02-08 and 2018-06-19.

This also confirms data is properly arranged and checks out, meaning the null values exist because there are no readings for those column values in the original data.

In [ ]:
# find only the columns that have missing values
null_columns = weather_data.columns[weather_data.isna().any()]
weather_data[null_columns].isna().sum()

In [ ]:
# Display the index for missing values
#weather_data[weather_data.isna().any(axis=1)].index

In [ ]:
# columns DataFrame with missing values
weather_data[weather_data.isna().any(axis=1)][null_columns]

In [ ]:
#fill null values with zeros
weather_data = weather_data.fillna(0)

In [ ]:
weather_data.isna().sum()

In [ ]:
# find duplicates
weather_data.duplicated().sum()

#### Weather Data Review <a class="anchor" id="DataReview"></a>

In [ ]:
weather_data.dtypes

In [ ]:
# frequencies for  Region column
weather_data.pivot_table(index= ['Region'], aggfunc='size')

#### Saving out the final C&P_Weather CSV File <a class="anchor" id="PreprocessedWeather"></a>

In [ ]:
final_file = "C&P_Weather.csv"
print("Saving file: '{}'".format(final_file))
weather_data.to_csv(final_file, index=False, encoding='utf-8')
print("File Saved...")

In [ ]:
# check DataFrame exported
df = pd.read_csv("P:\Wildfires_Australia\cfc_wildfireforecastforAustralia\C&P_Weather.csv")
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.Date.dtype.name

In [ ]:
df.isna().sum()